<a href="https://colab.research.google.com/github/LS-io/codeschoolmentorship/blob/main/TSS_JulyProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the 30 days of code challenge!
___
Feel free to add your code files you have had so far below.\
\
If you need(ed) to install any packages for python, you can do below in a code cell by running:
```
!pip install package-name --upgrade --quiet
```
*I add the --upgrade so it installs the latest package versions and --quiet so it doesn't spam all over the output cell, it will only show error messages

In [ ]:
!pip install *fun stuff* --upgrade --quiet